In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
pd.set_option('display.max_columns', None)


from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ECSE_552/Project'

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1t5tkKNNd877WyocXM37ISjMi0QHU1xjW/Project


In [ ]:
# Load temperature dataset
df = pd.read_csv('data/ON_demand_weather_17-20.csv')


# One hot encoding of categorical data for day of the week.
# No need to include Sunday. All 6 other days = false means it is Sunday.
df['Monday'] = (df['Day of Week'] == 0).astype(int)
df['Tuesday'] = (df['Day of Week'] == 1).astype(int)
df['Wednesday'] = (df['Day of Week'] == 2).astype(int)
df['Thursday'] = (df['Day of Week'] == 3).astype(int)
df['Friday'] = (df['Day of Week'] == 4).astype(int)
df['Saturday'] = (df['Day of Week'] == 5).astype(int)
#df['Sunday'] = (df['Day of Week'] == 6).astype(int)

# One hot encoding of categorical data for month data.
# No need to include december. All 11 other months = false means it is december.
df['January'] = (df['Month'] == 1).astype(int)
df['February'] = (df['Month'] == 2).astype(int)
df['March'] = (df['Month'] == 3).astype(int)
df['April'] = (df['Month'] == 4).astype(int)
df['May'] = (df['Month'] == 5).astype(int)
df['June'] = (df['Month'] == 6).astype(int)
df['July'] = (df['Month'] == 7).astype(int)
df['August'] = (df['Month'] == 8).astype(int)
df['September'] = (df['Month'] == 9).astype(int)
df['October'] = (df['Month'] == 10).astype(int)
df['November'] = (df['Month'] == 11).astype(int)
#df['December'] = (df['Month'] == 12).astype(int)



In [ ]:
# Function to apply time-differencing, see preprocessing .py

def time_differencing(df, column='Ontario Demand', lag=1, remove_first_rows=True):
    diff = df[column].diff(periods=lag)
    df['Demand Difference'] = diff.values

    if remove_first_rows:
        df = df.iloc[lag:]

    return df

In [ ]:
# Create sliding windows
def sliding_windows(data, window_size:int, num_of_output:int, flatten=False, output_window:int = 1):
  """
  Function to expand features to a sliding window
  ---------------------

  Inputs:
  data: Must be numpy by array.
  window_size: integer to specify the size of the sliding window
  num_of_output: integer that specifies the first "num_of_output" columns of data to taken as outputs
  If flatten is set to True:
    New number of features = number of features * sliding window size
    New number of time steps = number of time steps - sliding window size
  output_window: integer that specifies a forward sliding window to predict future value (default will only predict 1 time step)

  ---------
  Outputs:
  Returns features and labels sperately in a torch tensor.
  If flatten is set to False, the feature data structure will be "3 dimensional"
  1st dimension : time step
  2nd dimension : entry within the sliding window
  3rd dimension : feature 

  If flatten is set to True, the feature data structures will be 2 dimensional
  1st dimension : time step
  2nd dimension : each feature for each entry within the sliding window

  """
  if output_window < 1:
    raise Exception("Output window must be greater or equal than 1")
  
  x = []
  y = []
  for i in range(len(data)-window_size-1):
    feature_offset = i+window_size
    output_offset = feature_offset + output_window
    _x = data[i:feature_offset, :]
    _y = data[feature_offset:output_offset, :num_of_output]

    x.append(_x)
    y.append(_y)

  x = np.array(x)
  y = np.array(y)

  # Flatten output vector - This is always done, we don't want this behaviour
  # within if statement below
  y = y.reshape(y.shape[0], y.shape[1]*y.shape[2])

  # Flatten the feature and window dimensions
  if flatten:
    x = x.reshape(x.shape[0], x.shape[1]*x.shape[2])

  return (torch.from_numpy(x).float(), torch.from_numpy(y).float())


In [ ]:
df = df[["toronto_Temp (C)", "hamilton_Temp (C)", "kitchener_Temp (C)", "london_Temp (C)", "windsor_Temp (C)"]]

df = df.to_numpy()

In [ ]:
out_x, out_y  = sliding_windows(df,20,2, False, 3)

print(out_x.size())
print(out_y.size())

TypeError: ignored